In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
np.random.seed(72018)

def to_2d(array):
    return array.reshape(array.shape[0], -1)
  
def plot_exponential_data():
    data = np.exp(np.random.normal(size=1000))
    plt.hist(data)
    plt.show()
    return data
    
def plot_square_normal_data():
    data = np.square(np.random.normal(loc=5, size=1000))
    plt.hist(data)
    plt.show()
    return data

In [5]:
boston = pd.read_pickle('../data/boston_housing_clean.pickle')

boston_data = boston['dataframe']
boston_description = boston['description']

boston_data.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


In [6]:
y_col = "MEDV"

X = boston_data.drop(y_col, axis=1)
y = boston_data[y_col]

# Scaling the X data

In [7]:
from sklearn.preprocessing import StandardScaler

s = StandardScaler()
X_ss = s.fit_transform(X)

In [8]:
X_ss

array([[-0.41771335,  0.28482986, -1.2879095 , ..., -1.45900038,
         0.44105193, -1.0755623 ],
       [-0.41526932, -0.48772236, -0.59338101, ..., -0.30309415,
         0.44105193, -0.49243937],
       [-0.41527165, -0.48772236, -0.59338101, ..., -0.30309415,
         0.39642699, -1.2087274 ],
       ...,
       [-0.41137448, -0.48772236,  0.11573841, ...,  1.17646583,
         0.44105193, -0.98304761],
       [-0.40568883, -0.48772236,  0.11573841, ...,  1.17646583,
         0.4032249 , -0.86530163],
       [-0.41292893, -0.48772236,  0.11573841, ...,  1.17646583,
         0.44105193, -0.66905833]])

In [10]:
### Manually scaling the data and comparing with standard scaler
X2 = np.array(X)
man_transform = (X2-X2.mean(axis=0))/X2.std(axis=0)
np.allclose(man_transform, X_ss)

True

In [11]:
from sklearn.linear_model import LinearRegression

# Fit the model with unscaled data
lr = LinearRegression()
lr.fit(X, y)
print(lr.coef_)

[-1.07170557e-01  4.63952195e-02  2.08602395e-02  2.68856140e+00
 -1.77957587e+01  3.80475246e+00  7.51061703e-04 -1.47575880e+00
  3.05655038e-01 -1.23293463e-02 -9.53463555e-01  9.39251272e-03
 -5.25466633e-01]


In [12]:
#Fit the model with scaled data

lr_ss = LinearRegression()
lr_ss.fit(X_ss, y)
print(lr_ss.coef_)

[-0.92041113  1.08098058  0.14296712  0.68220346 -2.06009246  2.67064141
  0.02112063 -3.10444805  2.65878654 -2.07589814 -2.06215593  0.85664044
 -3.74867982]


In [13]:
pd.DataFrame(zip(X.columns, lr_ss.coef_)).sort_values(by=1)

,0,1
12,LSTAT,-3.748680
7,DIS,-3.104448
9,TAX,-2.075898
10,PTRATIO,-2.062156
4,NOX,-2.060092
0,CRIM,-0.920411
6,AGE,0.021121
2,INDUS,0.142967
3,CHAS,0.682203
11,B,0.856640


# Look at above's values:
A *high negative* value means that the prediction will *decrease* as the feature value increases
A *high positive* value means that the prediction will *increase* as the feature value increases  
In this case, coefficients LSTAT(-), DIS(-), RM(+) and RAD(+) are all the 'most impactful' features

In [17]:
from sklearn.linear_model import Lasso
from sklearn.preprocessing import PolynomialFeatures

# Create polynomial features
pf = PolynomialFeatures(degree=2, include_bias=False,)
X_pf = pf.fit_transform(X)

In [18]:
# Standardize the polynomial features
X_pf_ss = s.fit_transform(X_pf)

In [19]:
las = Lasso()
las.fit(X_pf_ss, y)
las.coef_ 

array([-0.        ,  0.        , -0.        ,  0.        , -0.        ,
        0.        , -0.        , -0.        , -0.        , -0.        ,
       -0.99073506,  0.        , -0.        , -0.        ,  0.        ,
       -0.        ,  0.06751971, -0.        , -0.        , -0.        ,
       -0.        , -0.        , -0.        , -0.        , -0.        ,
       -0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        , -0.        ,  0.        ,
       -0.        , -0.        , -0.        , -0.05012402, -0.        ,
       -0.        , -0.        , -0.        , -0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        , -0.        ,
       -0.        , -0.        , -0.        , -0.        , -0.        ,
       -0.        ,  0.        , -0.        ,  3.30027884, -0.  

In [20]:
las01 = Lasso(alpha = 0.1)
las01.fit(X_pf_ss, y)
print('sum of coefficients:', abs(las01.coef_).sum() )
print('number of coefficients not equal to 0:', (las01.coef_!=0).sum())

sum of coefficients: 26.17241511542675
number of coefficients not equal to 0: 23


In [21]:
las1 = Lasso(alpha = 1)
las1.fit(X_pf_ss, y)
print('sum of coefficients:',abs(las1.coef_).sum() )
print('number of coefficients not equal to 0:',(las1.coef_!=0).sum())

sum of coefficients: 8.472405227760158
number of coefficients not equal to 0: 7


In [22]:
from sklearn.metrics import r2_score

r2_score(y,las.predict(X_pf_ss))

0.7207000461229027

In [24]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_pf, y, test_size=0.3, random_state=72018)

In [25]:
X_train_s = s.fit_transform(X_train)
las.fit(X_train_s, y_train)
X_test_s = s.transform(X_test)
y_pred = las.predict(X_test_s)
r2_score(y_test, y_pred)

0.6780325981174931

In [26]:
X_train_s = s.fit_transform(X_train)
las01.fit(X_train_s, y_train)
X_test_s = s.transform(X_test)
y_pred = las01.predict(X_test_s)
r2_score(y_test, y_pred)

0.7999261342846063

# Now using Lasso with alpha = 0.001

In [28]:
las001 = Lasso(alpha = 0.001, max_iter=100000)
las001.fit(X_train_s, y_train)
y_pred = las001.predict(X_test_s)
r2_score(y_test, y_pred)

0.8847893236874582

In [35]:
lr = LinearRegression()
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)
r2_score(y_test, y_pred)

0.8689110469284284

Comparing the coefficients of Lasso with alpha = 0.001 and Linear Regression

In [33]:
print(f"-------Lasso model---------")
print(f"Sum of coefficients: {abs(las001.coef_).sum()}")
print(f"Number of coefficients non-zero: {(las001.coef_!=0).sum()}")
print(f"-------Linear Regression model---------")
print(f"Sum of coefficients: {abs(lr.coef_).sum()}")
print(f"Number of coefficients non-zero: {(lr.coef_!=0).sum()}")

-------Lasso model---------
Sum of coefficients: 435.57232290441243
Number of coefficients non-zero: 90
-------Linear Regression model---------
Sum of coefficients: 309.88577748756705
Number of coefficients non-zero: 104


# Using Ridge regression

In [37]:
from sklearn.linear_model import Ridge

r = Ridge(alpha=0.001)
X_train_s = s.fit_transform(X_train)
r.fit(X_train_s, y_train)
X_test_s = s.transform(X_test)
y_pred_r = r.predict(X_test_s)

print(f"Prediction value Ridge regression: {r2_score(y_test, y_pred_r)}")

r.coef_

Prediction value Ridge regression: 0.8702996266543306


array([  7.69158031,   9.88890473, -25.00525751,   5.31046818,
        -2.60993702,  14.95803313,  22.31375473, -22.8579177 ,
        27.76499232,  -1.56512089,  17.07749893,  21.85840549,
        11.57430607,   1.0501267 ,   0.43058525,  13.78571185,
         1.8138621 ,  -8.34497303,   4.99428114,  -3.56428247,
        -3.43620979, -16.33550823,  -7.04652168,   6.60858547,
        -1.47661   ,   4.68585754,  -1.30404236,  -0.05944784,
        -0.30379373, -12.84226172,   1.97669113,   1.08130677,
        -0.67550448,  -1.07931934,   4.49162494,  -4.26689794,
         4.67486611,  -1.28089201,   8.66602804,  -0.27373083,
        -8.11960573,  11.78297941,   6.53845776,   1.32299881,
         2.05956326,   0.89895643,   1.7894665 ,   4.74421976,
        -4.66441531,   5.31046818,  -3.23646667,  -8.66803392,
         0.97293208,   1.13586914,   0.29037187,  -1.63142731,
        -2.92599804,   2.92315095,  -0.73510358,  11.89603615,
         0.75403388,  -7.53119729,  18.30567198, -22.16

**Note**: Ridge doesn't zero out coefficients

In [39]:
y_pred = r.predict(X_pf_ss)
print(r2_score(y, y_pred))

# Lasso performs better than Ridge in this case
y_pred = las001.predict(X_pf_ss)
print(r2_score(y, y_pred))

0.9076091395031437
0.9103503442034482
